In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization,CuDNNLSTM,Activation,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import re
from keras import backend as K
import time
import gc
# To get learning rate


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))
train_length = len(train)

X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
# X_shiyi['weekDay'] = X_shiyi['time_day'] % 7 +1
print(X_shiyi.shape)

train = train.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
test = test.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')


X = pd.concat([train,test],sort=False)
print(X.shape)
X = X.reset_index(drop=True)

X_clean = pd.read_csv('../../data/original/cleaned_data_price_final.csv')
X_append = pd.read_pickle('../../data/nn_features/clean.pkl')



print(X.shape)
assert np.sum(X_clean['instance_id'].values != X['instance_id'].values) == 0
assert np.sum(X_append['instance_id'].values != X['instance_id'].values) == 0
# X['model'] = X_append['model_v3'].values


ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 45)
(1041674, 36)
(1041674, 36)


# PB 4236

In [3]:
# need_process_col = list(set(X.columns) - set(ignore_columns))
# X_ = X[need_process_col].copy()


# doc_col = ['user_tags','model']
# non_doc_col = [f for f in need_process_col if f not in doc_col]
# # doc_col = doc_col + []
# # non_doc_col = non_doc_col+['user_tags']
# counter = 0

# X_doc = X[doc_col].copy()
# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     X_doc[col] = X_doc[col].astype(str)
    
    

    

In [4]:
need_process_col = list(set(X.columns) - set(ignore_columns))
X_ = X[need_process_col].copy()


doc_col = ['user_tags','model']
non_doc_col = [f for f in need_process_col if f not in doc_col]
# doc_col = doc_col + []
# non_doc_col = non_doc_col+['user_tags']
counter = 0

X_doc = X[doc_col].copy()
for col in tqdm(non_doc_col):
#     col = 'creative_id'
    test_values = set(X_[col].iloc[train_length:].astype(str).unique())
    train_values = set(X_[col].iloc[:train_length].astype(str).unique())
    intersection = train_values.intersection(test_values)
    out_liyer = list(train_values.union(test_values) - intersection)
    if len(out_liyer) > 0:
#         print(len(out_liyer))
        out_liyer_mapping = pd.Series(index=list(out_liyer),data=1)
        filtered = (X_[col].astype(str).map(out_liyer_mapping) == 1)
#         print('col:{}, size:{}'.format(col,np.sum(filtered)))
        X_.loc[filtered,col] = np.nan
        
        
    X_[col] = le.fit_transform(X_[col].astype(str))
    X_[col] = col + '_'+X_[col].astype(str)
    
for col in tqdm(doc_col):
    X_doc[col] = X_doc[col].astype(str)
    
    

    

100%|██████████| 2/2 [00:00<00:00,  6.49it/s]


In [5]:
def get_tok(X,col,train_length,d_filter='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    X[col] = X[col].astype(str)

    tok_all=text.Tokenizer(num_words=X[col].nunique(),lower=False,filters=d_filter)
    tok_all.fit_on_texts(list(X[col].values))

    tok_train=text.Tokenizer(num_words=X[col].iloc[:train_length].nunique(),lower=False,filters=d_filter)
    tok_train.fit_on_texts(list(X[col].iloc[:train_length].values))

    tok_test=text.Tokenizer(num_words=X[col].iloc[train_length:].nunique(),lower=False,filters=d_filter)
    tok_test.fit_on_texts(list(X[col].iloc[train_length:].values))
    word_intersection = set(tok_train.word_index.keys()).intersection(set(tok_test.word_index.keys()))

    self_index = {}
    count = 1
    for word in word_intersection:
        self_index[word] = count
        count+=1
    self_index['unknown'] = count
    print('max index is: {}'.format(count))

    for word in tok_all.word_index.keys():
        tok_all.word_index[word] = self_index.get(word,count)
    return tok_all

# Combine columns

In [6]:
# # def comb_fe(X,cols,sep=' '):
# #     ret = X[cols[0]].astype(str).copy()
# #     for col in cols[1:]:
# #         ret = ret + sep + X[col].astype(str)
# #     return ret.values

# def process_slot(x):
#     x = re.sub(r'[-_]',' ',x)
#     x = x.split(' ')
#     ret = ['p{}_'.format(c)+x[c] for c in range(len(x))]
#     ret = ' '.join(ret)
#     return ret

# def comb_fe(X,cols,sep=' '):
#     def add_col_name(x,colName,splitter=' ' ,sep=' '):
#         ret = [colName+'_'+each for each in x.split(splitter)]
#         return sep.join(ret)
#     ret = pd.DataFrame()
#     if cols[0] == 'user_tags':
#         ret['comb'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#     else:
#         if cols[0] in original_name_col:
#             ret['comb'] = X[cols[0]].astype(str).copy()
#         else:
#             ret['comb'] = X[cols[0]].astype(str).apply(add_col_name,colName=cols[0],splitter=' ',sep=sep)

#     for col in tqdm(cols[1:]):
#         if col == 'user_tags':
#             ret['new_feature'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#         else:
#             if col in original_name_col:
#                 ret['new_feature'] = X[col].astype(str).copy()
#             else:
#                 ret['new_feature'] = X[col].astype(str).apply(add_col_name,colName=col,splitter=' ',sep=sep)
#         ret['comb'] =( ret['comb'] + sep + ret['new_feature']).values
#     return ret['comb'].values

# processed_col = []
# original_name_col = ['model','make','os','osv']
# # cob_col = [['model','make','os','osv']]
# # cob_col = [['model','make','os','osv'],['model','app_id','inner_slot_id','creative_width', 'creative_height']]
# # cob_col = [['model', 'osv']]
# cob_col=[]
# doc_col=['user_tags','model']
# # doc_col=['user_tags']
# X_doc = X[doc_col].copy()
# processed_col.extend(doc_col)
# for each in cob_col:
#     feature_name = '_'.join(each)
#     processed_col.extend(each)
#     doc_col.append(feature_name)
#     X_doc[feature_name] = comb_fe(X,each)
    
# non_doc_col = list(set(X.columns) - set(ignore_columns) - set(processed_col))
# X_ = X[non_doc_col].copy()

# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     if col=='inner_slot_id':
#         X_doc[col] = X_doc[col].astype(str).apply(process_slot)
#     else:
#         X_doc[col] = X_doc[col].astype(str)

In [7]:
# train_index = pickle.load(open(FILE.train_index.value,'rb'))
# holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))

# train_index = pickle.load(open('../../data/original/adversarial_train_index.pickle','rb'))
# holdout_index = pickle.load(open('../../data/original/adversarial_ho_index.pickle','rb'))

In [8]:
train_index = train.index
holdout_index = None

In [9]:
num_folds = 5
seed = 1001
train_index_list = []
val_index_list = []
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
for t,v in folds.split(train.loc[train_index],train.loc[train_index,'click']):
    train_index_list.append(train.loc[train_index].iloc[t].index)
    val_index_list.append(train.loc[train_index].iloc[v].index)
    
# check = []
# for i in val_index_list:
#     check.extend(list(i))
# assert len(set(check+list(holdout_index))) == len(train)

In [10]:
train_fold_y = {}
val_fold_y = {}
if holdout_index is not None:
    holdout_y = train.loc[holdout_index,'click'].values
else:
    holdout_y = None
for fold in range(num_folds):
    train_fold_y[fold] = train.loc[train_index_list[fold],'click'].values
    val_fold_y[fold] = train.loc[val_index_list[fold],'click'].values

In [11]:
info_dict = {}
train_all_dict = {}
train_fold_dict = {}

val_fold_dict = {}

holdout_input_dict = {}
test_input_dict = {}
maxlen = 1
prefix_input_nonDoc = 'input_'
prefix_input_Doc = 'input_rnn_'
for col in tqdm(non_doc_col):

    maxlen = 1
    tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters='@')
    tok.fit_on_texts(list(X_[col]))
    info_dict.update({prefix_input_nonDoc+col:{'tok':tok}})
    t = tok.texts_to_sequences(list(X_[col].iloc[:train_length].values))
    te = tok.texts_to_sequences(list(X_[col].iloc[train_length:].values))
    train_all_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    if holdout_index is not None:
        holdout_input_dict[prefix_input_nonDoc+col] = train_all_dict[prefix_input_nonDoc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_nonDoc+col: train_all_dict[prefix_input_nonDoc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_nonDoc+col:train_all_dict[prefix_input_nonDoc+col][list(val_index_list[fold])]})
        
sequence_size_dict = {}
for col in tqdm(doc_col):
    if col == 'user_tags':
        maxlen = 50
        tok = get_tok(X_doc,col=col,train_length=train_length,d_filter=',')
#         tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False, filters=',')
#         tok.fit_on_texts(list(X_doc[col]))

        
    elif col == 'model':
        maxlen = 15
#         tok = get_tok(X_doc,col=col,train_length=train_length)
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False,filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
        tok.fit_on_texts(list(X_doc[col]))
    else:
        maxlen = 15
#         tok = get_tok(X_doc,col=col,train_length=train_length)
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
        tok.fit_on_texts(list(X_doc[col]))
    info_dict.update({prefix_input_Doc+col:{'tok':tok}})
    sequence_size_dict[col] = maxlen
    t = tok.texts_to_sequences(list(X_doc[col].iloc[:train_length].values))
    te = tok.texts_to_sequences(list(X_doc[col].iloc[train_length:].values))
    train_all_dict[prefix_input_Doc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_Doc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    if holdout_index is not None:
        holdout_input_dict[prefix_input_Doc+col] = train_all_dict[prefix_input_Doc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_Doc+col: train_all_dict[prefix_input_Doc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_Doc+col:train_all_dict[prefix_input_Doc+col][list(val_index_list[fold])]})

  0%|          | 0/2 [00:00<?, ?it/s]

max index is: 1313


100%|██████████| 2/2 [00:56<00:00, 28.32s/it]


In [12]:
tok = info_dict['input_rnn_model']['tok']

In [13]:
tok.word_index

{'OPPOR9PlusmA': 2101,
 '25252C3': 4893,
 'H32': 5863,
 'NX40X': 9315,
 'K10e70': 801,
 'WDB': 5701,
 'Cong': 2870,
 'T350': 5415,
 'D5503': 8916,
 'KT40Q': 6695,
 'bodu': 8353,
 'TCL580': 8752,
 'A6600': 4789,
 'mlledn9': 6092,
 'Y65': 3593,
 'G0245D': 2706,
 '850M': 4236,
 'A10S': 6774,
 'ovvi': 8813,
 '20Y31': 1185,
 'V': 343,
 'x18': 7975,
 '20G628': 2636,
 '2525252525252BPlus': 810,
 'A73': 87,
 '252BA33m': 2889,
 'vivoX710L': 5866,
 '2525252525252BF7': 7518,
 'BD': 5519,
 '252525252525252BA3010': 6205,
 'U5483': 3093,
 '2525252BY71A': 2254,
 '8718': 990,
 'G3509I': 3158,
 'S9L': 963,
 '252525252BY83A': 4710,
 '25252BA8': 5776,
 '4W': 635,
 'NT1': 2902,
 'D58X': 8086,
 'I9128': 4178,
 'qh106': 7535,
 'iPhone4.1': 4302,
 'G950U': 4359,
 'y51a': 1796,
 'a0001': 4110,
 'a03': 2444,
 '2525252BMAX': 1729,
 'k52t38': 6766,
 '2525252525252BK8300': 8656,
 '2BPlust': 5645,
 'XIAOMI': 5273,
 '2525252525252BA5000': 4357,
 '25252525252BX6A': 2843,
 'ONEPLUS': 166,
 '25252525252B6X': 2239,
 'a

# Build NN model only use model

# 0.41479

In [14]:
def get_nn_model(cols,doc_cols=[],numu_cols=[]):
    """
    cols, used to do ebd and dense layers
    doc_cols: used to do rnn
    there can be overlaps
    """
    input_list = []
    concat_list = []
    numu_list = []
    for col in cols:
        max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
#         max_feature = len(set(info_dict[prefix_input_nonDoc+col]['tok'].index_word.values()))
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=1,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        concat_list.append(x)
    for col in doc_cols:
       
        max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
#         max_feature = len(set(info_dict[prefix_input_Doc+col]['tok'].index_word.values()))
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
        if col == 'model':
            embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
            x = SpatialDropout1D(0.5)(embed_layer)
            x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
            x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
#             x2 = Conv1D(99, kernel_size = 2, padding = "valid", kernel_initializer = "glorot_uniform")(x)
            x_aveP = GlobalAveragePooling1D()(x)
            x_maxP = GlobalMaxPooling1D()(x)
#             x_aveP2 = GlobalAveragePooling1D()(x2)
#             x_maxP2 = GlobalMaxPooling1D()(x2)
            x = concatenate([x_aveP,x_maxP])
        else:
            embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
            x = SpatialDropout1D(0.5)(embed_layer)
            x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
            x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
            x_aveP = GlobalAveragePooling1D()(x)
            x_maxP = GlobalMaxPooling1D()(x)
            x = concatenate([x_aveP,x_maxP])
        input_list.append(cur_input)
        concat_list.append(x)

    if len(numu_cols) > 0:
        print('add numu...')
        nu_shape = len(numu_cols)
        cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
        x = BatchNormalization()(cur_input)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        input_list.append(cur_input)
        numu_list.append(x)
       
    if len(concat_list) > 1:
        x = concatenate(concat_list)
#     x = BatchNormalization()(x)
    

    if len(numu_list)>0:
        x = concatenate([x]+numu_list)
#         x = BatchNormalization()(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model


In [15]:

# def get_nn_model(cols,doc_cols=[],numu_cols=[]):
#     """
#     cols, used to do ebd and dense layers
#     doc_cols: used to do rnn
#     there can be overlaps
#     """
#     input_list = []
#     concat_list = []
#     numu_list = []
#     for col in cols:
#         max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=1,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_'+col)(cur_input)
#         embed_layer = SpatialDropout1D(0.5)(embed_layer)
#         x = Flatten()(embed_layer)
#         input_list.append(cur_input)
#         concat_list.append(x)
#     for col in doc_cols:
#         max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         input_shape = sequence_size_dict[col]
#         cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=input_shape,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_rnn_'+col)(cur_input)
#         x = SpatialDropout1D(0.5)(embed_layer)
#         x = Bidirectional(CuDNNGRU(256, return_sequences=True))(x)
#         x = Conv1D(128, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
#         x_aveP = GlobalAveragePooling1D()(x)
#         x_maxP = GlobalMaxPooling1D()(x)
#         x = concatenate([x_aveP, x_maxP]) 
#         input_list.append(cur_input)
#         concat_list.append(x)
# #         concat_list.append(x)
#     if len(numu_cols) > 0:
#         print('add numu...')
#         nu_shape = len(numu_cols)
#         cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
#         x = BatchNormalization()(cur_input)
#         x = Dense(128, activation='relu')(x)
#         x = Dropout(0.2)(x)
#         input_list.append(cur_input)
#         numu_list.append(x)
       
#     if len(concat_list) > 1:
#         x = concatenate(concat_list)
# #     x = BatchNormalization()(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.2)(x)

#     if len(numu_list)>0:
#         x = concatenate([x]+numu_list)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)

#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model(input_list, preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
#     return model

In [16]:
def train_each_fold(input_train_dict,input_val_dict,y_train,y_val,cols,doc_col=[],tolerance=30,train_batch_size=5000):
    model = get_nn_model(cols,doc_col)
    cur_to = 0
    best_logloss = None
    best_weights = None
    count = 0
    base_lr = 0.001
    while True:
        model.fit(input_train_dict, y_train, 
                  batch_size=train_batch_size, 
                  epochs=1,
                  verbose=1,
                  shuffle=True,
                  )
        preds = model.predict(input_val_dict,5000,verbose=1)
        logloss = log_loss(y_val,preds)
        roc = roc_auc_score(y_val,preds)
        print(logloss)
        print(roc)
        if best_logloss is None:
            best_logloss = logloss
            best_weights = model.get_weights()
        else:
            if best_logloss > logloss:
                best_logloss = logloss
                best_weights = model.get_weights()
                cur_to = 0
            else:
                cur_to +=1
        if cur_to == tolerance:
            break
        print('best logloss is: {}'.format(best_logloss))
        print('remainning trial is: {}/{}'.format(cur_to,tolerance))
        print('total epoch trained: {}'.format(count))
        count+=1
    model.set_weights(best_weights)
    return model
    

In [17]:
def nn_K_fold(train_fold_dict,
              val_fold_dict,
              train_fold_y,
              val_fold_y,
              test_input_dict,
              val_index_list,
              train_df,
              test_df,
              pred_col_name = 'predicted_score',
              holdout_input_dict=None,
              holdout_y=None,
              holdout_index_list=None,
              nondoc_cols=[],
              doc_cols=[],
              tolerance=30,
              train_batch_size=5000,
              preds_batch=5000):
    """
    train_fold_dict: format, key - foldNum, value - nn input 
    train_fold_y: label for train fold, fotmat, key -foldNum, value - label
    val_fold_dict: format, key - foldNum, value - nn input 
    val_fold_y: label for val fold, fotmat, key -foldNum, value - label
    test_input_dict: format, key - foldNum, value - nn input 
    holdout_input_dict: Noneable, if none, not using holdout
    holdout_y: Noneable, label for holdout
    cols: all cols used to do ebd
    doc_cols, cols that are used to do rnn
    val_index_list: cv, going to predict and generate oof
    holdout_index_list: going to predict on each fold
    train_df: 'dataframe which only has id columns, which will be used to store oof prediction'
    test_df: 'dataframe which only has id columns, which will be used to store test prediction'
    """
    train_df = train_df.copy()
    test_df = test_df.copy()
    n_fold = len(train_fold_dict)
    test_preds_list = []
    val_score_list = []
    hold_out_preds_list = []
    holdout_score_list = []
    train_score_list = []
    cv_score_list = []
    train_df[pred_col_name] = np.nan
    test_df[pred_col_name] = np.nan
    
    for fold in range(n_fold):
        print('start fold {}...'.format(fold))
        model = train_each_fold(train_fold_dict[fold],
                                val_fold_dict[fold],
                                train_fold_y[fold],
                                val_fold_y[fold],
                                nondoc_cols,
                                doc_cols,
                                tolerance=tolerance,
                                train_batch_size=train_batch_size)
        train_preds =  model.predict(train_fold_dict[fold],preds_batch,verbose=1)
        val_preds = model.predict(val_fold_dict[fold],preds_batch,verbose=1)
        train_loss = log_loss(train_fold_y[fold],train_preds)
        val_loss = log_loss(val_fold_y[fold],val_preds)
        train_df.loc[val_index_list[fold],pred_col_name] = val_preds
        test_preds = model.predict(test_input_dict,preds_batch,verbose=1)
        test_preds_list.append(test_preds)
        val_score_list.append(val_loss)
        train_score_list.append(train_loss)
        print('Fold {} finish! val loss: {}.'.format(fold,val_loss))
        if holdout_index_list is not None:
            ho_preds = model.predict(holdout_input_dict,preds_batch,verbose=1)
            ho_loss = log_loss(holdout_y,ho_preds)
            ho_roc = roc_auc_score(holdout_y,ho_preds)
            holdout_score_list.append(ho_loss)
            hold_out_preds_list.append(ho_preds)
            print('hold out loss: {}'.format(ho_loss))
        del model
        gc.collect()
        time.sleep(5)
            
    print('finish training... calculating evl matrix')
    test_preds_list = np.array(test_preds_list)
    hold_out_preds_list = np.array(hold_out_preds_list)
    test_preds_final = np.mean(test_preds_list,axis=0)
    cv_score_mean = np.mean(val_score_list)
    train_score_mean = np.mean(train_score_list)
    test_df[pred_col_name] = test_preds_final
    print('cv mean is: {}'.format(cv_score_mean))
    if holdout_index_list is not None:
        ho_preds_final = np.mean(hold_out_preds_list,axis=0)
        ho_loss_overall = log_loss(holdout_y,ho_preds_final)
        ho_roc_overall = roc_auc_score(holdout_y,ho_preds_final)
        train_df.loc[holdout_index_list,pred_col_name] = ho_preds_final
        print('holdout loss overall is: {}'.format(ho_loss_overall))
        print('holdout roc overall is: {}'.format(ho_roc_overall))
        return train_df,test_df,cv_score_mean,ho_loss_overall,train_score_mean
    else:
        return train_df,test_df,cv_score_mean,0.0,train_score_mean
    
        

In [ ]:
train_df = train[['instance_id']].copy()
test_df = test[['instance_id']].copy()

train_save,test_save,cv_,ho_,ta_ = nn_K_fold(train_fold_dict,
                                      val_fold_dict,
                                      train_fold_y,
                                      val_fold_y,
                                      test_input_dict,
                                      val_index_list,
                                      train_df,
                                      test_df,
                                      pred_col_name = 'predicted_score',
                                      holdout_input_dict=holdout_input_dict,
                                      holdout_y=holdout_y,
                                      holdout_index_list=holdout_index,
                                      nondoc_cols=non_doc_col,
                                      doc_cols=doc_col,
                                      tolerance=30,
                                      preds_batch=5000)

start fold 0...
Epoch 1/1
200331/200331 [==============================] - 2s 9us/step
0.42288232047659496
0.7567416312670407
best logloss is: 0.42288232047659496
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4204427296863615
0.7622690396288728
best logloss is: 0.4204427296863615
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41885247837939293
0.7642076799163784
best logloss is: 0.41885247837939293
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.4179401273063227
0.764901354172986
best logloss is: 0.4179401273063227
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4178372056226309
0.7657034102987207
best logloss is: 0.4178372056226309
remainning trial is: 0/30
total epoch trained: 4
Epoch 1/1
2

200331/200331 [==============================] - 2s 8us/step
0.4156544814857647
0.7687897198545304
best logloss is: 0.4156544814857647
remainning trial is: 0/30
total epoch trained: 28
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41555533642452036
0.7689749232756979
best logloss is: 0.41555533642452036
remainning trial is: 0/30
total epoch trained: 29
Epoch 1/1
200331/200331 [==============================] - 2s 7us/step
0.41576672048786567
0.7685199108082379
best logloss is: 0.41555533642452036
remainning trial is: 1/30
total epoch trained: 30
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4158680669955739
0.7685493834260835
best logloss is: 0.41555533642452036
remainning trial is: 2/30
total epoch trained: 31
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.415234813785679
0.7690170746564446
best logloss is: 0.415234813785679
remainning trial is: 0/30
total epoch trained: 32
Epoch 1/1
200331/200331 [=======

200331/200331 [==============================] - 2s 8us/step
0.4154945334712687
0.7692848570148152
best logloss is: 0.4148918328301579
remainning trial is: 2/30
total epoch trained: 57
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.4149831201263897
0.7695237792181274
best logloss is: 0.4148918328301579
remainning trial is: 3/30
total epoch trained: 58
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4150787385570713
0.7697496594101487
best logloss is: 0.4148918328301579
remainning trial is: 4/30
total epoch trained: 59
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.4153239939515792
0.7693779880122937
best logloss is: 0.4148918328301579
remainning trial is: 5/30
total epoch trained: 60
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41521213087537745
0.7695573811039453
best logloss is: 0.4148918328301579
remainning trial is: 6/30
total epoch trained: 61
Epoch 1/1
200331/200331 [=========

200331/200331 [==============================] - 2s 7us/step
0.4151502359314743
0.7691822999644132
best logloss is: 0.4147729287110368
remainning trial is: 20/30
total epoch trained: 85
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4150034297151405
0.7694738273170857
best logloss is: 0.4147729287110368
remainning trial is: 21/30
total epoch trained: 86
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4150508543450646
0.7696737994716765
best logloss is: 0.4147729287110368
remainning trial is: 22/30
total epoch trained: 87
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4150572583281296
0.7696432255956143
best logloss is: 0.4147729287110368
remainning trial is: 23/30
total epoch trained: 88
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41521612094616384
0.7692187414219476
best logloss is: 0.4147729287110368
remainning trial is: 24/30
total epoch trained: 89
Epoch 1/1
200331/200331 [====

200331/200331 [==============================] - 1s 7us/step
0.4150337142273582
0.7704050383736492
best logloss is: 0.4150337142273582
remainning trial is: 0/30
total epoch trained: 17
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4152266443965976
0.7701898444824684
best logloss is: 0.4150337142273582
remainning trial is: 1/30
total epoch trained: 18
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4148163671741365
0.7706054726165323
best logloss is: 0.4148163671741365
remainning trial is: 0/30
total epoch trained: 19
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41480642144099283
0.7704931333919418
best logloss is: 0.41480642144099283
remainning trial is: 0/30
total epoch trained: 20
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.4149799105107754
0.7706223006973385
best logloss is: 0.41480642144099283
remainning trial is: 1/30
total epoch trained: 21
Epoch 1/1
200331/200331 [=======

...

200329/200329 [==============================] - 2s 8us/step
0.4182703198641363
0.7641188749165316
best logloss is: 0.4182703198641363
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4197131881660482
0.7639289549763513
best logloss is: 0.4182703198641363
remainning trial is: 1/30
total epoch trained: 4
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.41798184685409856
0.7652305780841842
best logloss is: 0.41798184685409856
remainning trial is: 0/30
total epoch trained: 5
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.41822147400363263
0.7652595217751391
best logloss is: 0.41798184685409856
remainning trial is: 1/30
total epoch trained: 6
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.418111040202696
0.7655045769226463
best logloss is: 0.41798184685409856
remainning trial is: 2/30
total epoch trained: 7
Epoch 1/1
200329/200329 [===========

200329/200329 [==============================] - 2s 8us/step
0.4164480255804906
0.7679789157264052
best logloss is: 0.41644053845924034
remainning trial is: 1/30
total epoch trained: 31
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.41595570128285775
0.7684148071091055
best logloss is: 0.41595570128285775
remainning trial is: 0/30
total epoch trained: 32
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.41621153426565294
0.7680074285652476
best logloss is: 0.41595570128285775
remainning trial is: 1/30
total epoch trained: 33
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4163761762724937
0.7682617134719809
best logloss is: 0.41595570128285775
remainning trial is: 2/30
total epoch trained: 34
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.4161689394481789
0.7680674565796044
best logloss is: 0.41595570128285775
remainning trial is: 3/30
total epoch trained: 35
Epoch 1/1
200329/200329 [===

200329/200329 [==============================] - 1s 7us/step
0.4162420304569117
0.7683400780384535
best logloss is: 0.4157488031994191
remainning trial is: 6/30
total epoch trained: 60
Epoch 1/1
200329/200329 [==============================] - 2s 7us/step
0.41618308299547524
0.7685680447813209
best logloss is: 0.4157488031994191
remainning trial is: 7/30
total epoch trained: 61
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.41612478090180494
0.7685016966888836
best logloss is: 0.4157488031994191
remainning trial is: 8/30
total epoch trained: 62
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.41583024603536556
0.769058751658873
best logloss is: 0.4157488031994191
remainning trial is: 9/30
total epoch trained: 63
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4161213243595312
0.7687720957465884
best logloss is: 0.4157488031994191
remainning trial is: 10/30
total epoch trained: 64
Epoch 1/1
200329/200329 [=======

200329/200329 [==============================] - 1s 7us/step
0.41906854590046366
0.7634572877520628
best logloss is: 0.41906854590046366
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.41830303145985814
0.7641824767085734
best logloss is: 0.41830303145985814
remainning trial is: 0/30
total epoch trained: 4
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.41802284766006576
0.7644272035281299
best logloss is: 0.41802284766006576
remainning trial is: 0/30
total epoch trained: 5
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.4178148892996198
0.7652097458636962
best logloss is: 0.4178148892996198
remainning trial is: 0/30
total epoch trained: 6
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.4179616539167564
0.7651761708062402
best logloss is: 0.4178148892996198
remainning trial is: 1/30
total epoch trained: 7
Epoch 1/1
200329/200329 [=========

200329/200329 [==============================] - 2s 7us/step
0.41624165804234114
0.7684920062366696
best logloss is: 0.41624165804234114
remainning trial is: 0/30
total epoch trained: 31
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.416443542432226
0.7679244581654115
best logloss is: 0.41624165804234114
remainning trial is: 1/30
total epoch trained: 32
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.41644725979710606
0.767979973628511
best logloss is: 0.41624165804234114
remainning trial is: 2/30
total epoch trained: 33
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4169289580057979
0.7679928897931876
best logloss is: 0.41624165804234114
remainning trial is: 3/30
total epoch trained: 34
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4166950061862912
0.7679743016691785
best logloss is: 0.41624165804234114
remainning trial is: 4/30
total epoch trained: 35
Epoch 1/1
200329/200329 [=====

In [ ]:
def save_oof(train_df,test_df,cv,ta,ho=None,file_name='',path='../../data/nn_ebd/'):
    try:
        report = pd.read_csv(path+'report.csv')
    except:
        print('no report found! generate a new one!')
        report = pd.DataFrame()
    new_record = pd.DataFrame({'ho':[ho],'cv':[cv],'train_mean':[ta],'file':[file_name]})
    report = pd.concat([report,new_record],sort=False)
    train_df.to_pickle(path+'train/'+file_name+'.pkl')
    print(train_df.shape)
    test_df.to_csv(path+'test/'+file_name+'.csv',index=False)
    print(test_df.shape)
    report.to_csv(path+'report.csv',index=False)
    print('done!')
    

In [ ]:
save_oof(train_save,test_save,cv_,ta_,ho_,file_name='oldModel_RNNModelTag_ebdNANExclude_changeModelFilter_noholdout')

In [ ]:
name_dict = {'new_modelMakeOsOSV_rnnEBD':
             [['model','make','os','osv']],
             'new_modelMakeOsOSV+modelAppidInnerslotCreateWCreativeH_rnnEBD':
             [['model','make','os','osv'],['model','app_id','inner_slot_id','creative_width', 'creative_height']],
             'new_modelOSV':[['model', 'osv']],'new_model_onlyuTagRNN_100patience':[],}


In [ ]:
tok=text.Tokenizer(num_words=100,lower=False)
tok.fit_on_texts(list(['9.2.1 Iphone']))
tok.word_index

In [ ]:
tok=text.Tokenizer(num_words=100,lower=False,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tok.fit_on_texts(list(['9.2.1 Iphone']))
tok.word_index

In [ ]:
tok.word_index